In [41]:
import numpy as np
a = np.random.random(10**7)
b = np.random.random(10**7)

In [42]:
!pip install numexpr

In [43]:
import numexpr as ne

In [44]:
%%timeit
np.cos(a) + np.sin(b)

557 ms ± 97.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
!pip install bottleneck

In [46]:
import bottleneck as bn

In [47]:
import pandas as pd

In [48]:
!pip install pandarallel
!pip install mapply
!pip install parallel-pandas
!pip install swifter

# Практическое задание 5. Polars

## Считывание файла

# Новый раздел

In [49]:
import polars as pl

In [50]:
df = pl.read_csv("train.csv")

## Вывод основной информации о датасете

In [51]:
print(df.describe())
print(df.dtypes)
print(df.null_count())
print(df.mean())

shape: (9, 13)
┌────────────┬─────────────┬──────────┬──────────┬───┬───────────┬───────────┬───────┬──────────┐
│ statistic  ┆ PassengerId ┆ Survived ┆ Pclass   ┆ … ┆ Ticket    ┆ Fare      ┆ Cabin ┆ Embarked │
│ ---        ┆ ---         ┆ ---      ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---   ┆ ---      │
│ str        ┆ f64         ┆ f64      ┆ f64      ┆   ┆ str       ┆ f64       ┆ str   ┆ str      │
╞════════════╪═════════════╪══════════╪══════════╪═══╪═══════════╪═══════════╪═══════╪══════════╡
│ count      ┆ 891.0       ┆ 891.0    ┆ 891.0    ┆ … ┆ 891       ┆ 891.0     ┆ 204   ┆ 889      │
│ null_count ┆ 0.0         ┆ 0.0      ┆ 0.0      ┆ … ┆ 0         ┆ 0.0       ┆ 687   ┆ 2        │
│ mean       ┆ 446.0       ┆ 0.383838 ┆ 2.308642 ┆ … ┆ null      ┆ 32.204208 ┆ null  ┆ null     │
│ std        ┆ 257.353842  ┆ 0.486592 ┆ 0.836071 ┆ … ┆ null      ┆ 49.693429 ┆ null  ┆ null     │
│ min        ┆ 1.0         ┆ 0.0      ┆ 1.0      ┆ … ┆ 110152    ┆ 0.0       ┆ A10   ┆ C        │
│ 25%

## Количество пассажиров каждого класса

In [52]:
pclass_column = df.get_column("Pclass")
pclass_counts = pclass_column.value_counts().sort("Pclass")
print(pclass_counts)

shape: (3, 2)
┌────────┬───────┐
│ Pclass ┆ count │
│ ---    ┆ ---   │
│ i64    ┆ u32   │
╞════════╪═══════╡
│ 1      ┆ 216   │
│ 2      ┆ 184   │
│ 3      ┆ 491   │
└────────┴───────┘


## Количество выживших мужчин и женщин

In [53]:
survived_counts = df.group_by("Sex").agg(
    pl.col("Survived").sum().alias("Выжившие")
)
print(survived_counts)

shape: (2, 2)
┌────────┬──────────┐
│ Sex    ┆ Выжившие │
│ ---    ┆ ---      │
│ str    ┆ i64      │
╞════════╪══════════╡
│ male   ┆ 109      │
│ female ┆ 233      │
└────────┴──────────┘


##  Пассажиры старше 44 лет

In [54]:
older_passengers = df.filter(pl.col("Age") > 44)
print(older_passengers.head(5))

shape: (5, 12)
┌─────────────┬──────────┬────────┬──────────────────┬───┬────────────┬─────────┬───────┬──────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name             ┆ … ┆ Ticket     ┆ Fare    ┆ Cabin ┆ Embarked │
│ ---         ┆ ---      ┆ ---    ┆ ---              ┆   ┆ ---        ┆ ---     ┆ ---   ┆ ---      │
│ i64         ┆ i64      ┆ i64    ┆ str              ┆   ┆ str        ┆ f64     ┆ str   ┆ str      │
╞═════════════╪══════════╪════════╪══════════════════╪═══╪════════════╪═════════╪═══════╪══════════╡
│ 7           ┆ 0        ┆ 1      ┆ McCarthy, Mr.    ┆ … ┆ 17463      ┆ 51.8625 ┆ E46   ┆ S        │
│             ┆          ┆        ┆ Timothy J        ┆   ┆            ┆         ┆       ┆          │
│ 12          ┆ 1        ┆ 1      ┆ Bonnell, Miss.   ┆ … ┆ 113783     ┆ 26.55   ┆ C103  ┆ S        │
│             ┆          ┆        ┆ Elizabeth        ┆   ┆            ┆         ┆       ┆          │
│ 16          ┆ 1        ┆ 2      ┆ Hewlett, Mrs.    ┆ … ┆ 248706     ┆ 16.0

# Pandas

## Считывае файл с помощью pandas

In [55]:
df = pd.read_csv('train.csv')

## Расчет среднего возраста и стандартного отклонения

In [56]:
mean = bn.nanmean(df['Age'].values)
std = bn.nanstd(df['Age'].values)

print(f"Средний возраст: {mean}")
print(f"Стандартное отклонение возраста: {std}")

Средний возраст: 29.69911764705882
Стандартное отклонение возраста: 14.516321150817317


## Создание нового столбца

In [57]:
df['Fare_new'] = df['Fare'].apply(lambda x: x * 1.3)

# Оптимизация типов pandas

## Считывание файла

In [58]:
df = pd.read_csv('Housing.csv')

## Анализ текущих типов данных и потребления памяти

In [61]:
initial_memory = df.memory_usage(deep=True).sum()
print(f"Исходное использование памяти: {initial_memory / 1024 ** 2}")

print("\nТекущие типы данных:")
print(df.dtypes)

Исходное использование памяти: 0.24582290649414062

Текущие типы данных:
price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object
